In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession,types
spark=SparkSession.builder.master("local").appName("new").getOrCreate()
sc=spark.sparkContext


In [6]:
data=spark.read.text(r"C:\Users\2087964\OneDrive - Cognizant\Desktop\input.txt")

In [7]:
data.show(truncate=0)

+---------------------+
|value                |
+---------------------+
|Name~|Age            |
|Azarudeen, Shahul~|25|
|Michel, Clarke~|26   |
|Virat, Kohli~|28     |
|Andrew, Simond~|28   |
|George, Bush~|37     |
|Flintoff, David~|12  |
|Adam, James~|20      |
+---------------------+



In [10]:
header=data.first()[0]

In [12]:
schema=header.split("~|")

In [14]:
data.filter(data["value"]!=header).show()

+--------------------+
|               value|
+--------------------+
|Azarudeen, Shahul...|
|  Michel, Clarke~|26|
|    Virat, Kohli~|28|
|  Andrew, Simond~|28|
|    George, Bush~|37|
| Flintoff, David~|12|
|     Adam, James~|20|
+--------------------+



In [15]:
data.filter(data["value"]!=header).rdd.map(lambda x:x[0].split("~|")).collect()

[['Azarudeen, Shahul', '25'],
 ['Michel, Clarke', '26'],
 ['Virat, Kohli', '28'],
 ['Andrew, Simond', '28'],
 ['George, Bush', '37'],
 ['Flintoff, David', '12'],
 ['Adam, James', '20']]

In [22]:
dataN=data.filter(data["value"]!=header).rdd.map(lambda x:x[0].split("~|")).toDF(schema)

In [23]:
dataN.show()

+-----------------+---+
|             Name|Age|
+-----------------+---+
|Azarudeen, Shahul| 25|
|   Michel, Clarke| 26|
|     Virat, Kohli| 28|
|   Andrew, Simond| 28|
|     George, Bush| 37|
|  Flintoff, David| 12|
|      Adam, James| 20|
+-----------------+---+



In [24]:
rdd1=dataN.rdd

In [29]:
rdd1.collect()

[Row(Name='Azarudeen, Shahul', Age='25'),
 Row(Name='Michel, Clarke', Age='26'),
 Row(Name='Virat, Kohli', Age='28'),
 Row(Name='Andrew, Simond', Age='28'),
 Row(Name='George, Bush', Age='37'),
 Row(Name='Flintoff, David', Age='12'),
 Row(Name='Adam, James', Age='20')]

In [47]:
dataF=rdd1.map(lambda x:x[0].split(",")).map(lambda x:x[0]+" "+x[1]).map(lambda x:x.split(",")).toDF(["Name"])

In [49]:
dataN.show()

+-----------------+---+
|             Name|Age|
+-----------------+---+
|Azarudeen, Shahul| 25|
|   Michel, Clarke| 26|
|     Virat, Kohli| 28|
|   Andrew, Simond| 28|
|     George, Bush| 37|
|  Flintoff, David| 12|
|      Adam, James| 20|
+-----------------+---+



In [52]:
dataF.show()

+-----------------+
|             Name|
+-----------------+
|Azarudeen  Shahul|
|   Michel  Clarke|
|     Virat  Kohli|
|   Andrew  Simond|
|     George  Bush|
|  Flintoff  David|
|      Adam  James|
+-----------------+



In [55]:
dataN=dataN.drop("Name")

In [56]:
dataN.show()

+---+
|Age|
+---+
| 25|
| 26|
| 28|
| 28|
| 37|
| 12|
| 20|
+---+



In [71]:
from pyspark.sql.functions import monotonically_increasing_id 

data_index = dataN.select("*").withColumn("id", monotonically_increasing_id())

In [72]:
data_index.show()

+---+---+
|Age| id|
+---+---+
| 25|  0|
| 26|  1|
| 28|  2|
| 28|  3|
| 37|  4|
| 12|  5|
| 20|  6|
+---+---+



In [73]:
dataN=dataF.select("*").withColumn("id",monotonically_increasing_id())

In [74]:
dataN.show()

+-----------------+---+
|             Name| id|
+-----------------+---+
|Azarudeen  Shahul|  0|
|   Michel  Clarke|  1|
|     Virat  Kohli|  2|
|   Andrew  Simond|  3|
|     George  Bush|  4|
|  Flintoff  David|  5|
|      Adam  James|  6|
+-----------------+---+



In [80]:
final_data=dataN.join(data_index,on="id").drop("id").sort("Name")

In [82]:
final_data.show()

+-----------------+---+
|             Name|Age|
+-----------------+---+
|      Adam  James| 20|
|   Andrew  Simond| 28|
|Azarudeen  Shahul| 25|
|  Flintoff  David| 12|
|     George  Bush| 37|
|   Michel  Clarke| 26|
|     Virat  Kohli| 28|
+-----------------+---+

